### Extract Knives

In [ ]:
import os
import cv2

image_dir = "/home/group02-f24/Documents/Khalil/CV/Traditional_CV/Heavy Weapon Dataset.v1i.yolov11/train/images"
label_dir = "/home/group02-f24/Documents/Khalil/CV/Traditional_CV/Heavy Weapon Dataset.v1i.yolov11/train/labels"
output_dir = "/home/group02-f24/Documents/Khalil/CV/Traditional_CV/HeavyWeapons_cropped"

os.makedirs(output_dir, exist_ok=True)

for label_filename in os.listdir(label_dir):
    if not label_filename.endswith(".txt"):
        continue
    label_path = os.path.join(label_dir, label_filename)
    image_name = os.path.splitext(label_filename)[0] + ".jpg"
    image_path = os.path.join(image_dir, image_name)

    if not os.path.exists(image_path):
        continue

    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    
    with open(label_path, "r") as file:
        for i, line in enumerate(file):
            parts = line.strip().split()
            if len(parts) != 5:
                continue

            class_id, x_center, y_center, w, h = map(float, parts)

            
            x_center *= width
            y_center *= height
            w *= width
            h *= height

            x1 = int(x_center - w / 2)
            y1 = int(y_center - h / 2)
            x2 = int(x_center + w / 2)
            y2 = int(y_center + h / 2)

            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(width, x2)
            y2 = min(height, y2)

            cropped = image[y1:y2, x1:x2]
            output_filename = f"{os.path.splitext(image_name)[0]}_weapon_{i}.jpg"
            output_path = os.path.join(output_dir, output_filename)
            cv2.imwrite(output_path, cropped)

print("Done. Cropped knife images saved to 'HeavyWeapons' folder.")


Done. Cropped knife images saved to 'HeavyWeapons' folder.


### Feature Extraction

#### Histogram of Oriented Gradients (HOG)

In [2]:
import os
import pandas as pd

data = []
for label, folder in [(1, '/home/group02-f24/Documents/Khalil/CV/Traditional_CV/Weapons'), (0, 'no_weapons')]:
    for fn in os.listdir(folder):
        if fn.lower().endswith(('.jpg','.png','.jpeg')):
            data.append([os.path.join(folder, fn), label])
df = pd.DataFrame(data, columns=['filepath','label'])


#### Color Histogram

In [ ]:
from skimage.feature import hog
from skimage import io
import numpy as np

TARGET_SIZE = (128, 128)  

def extract_hog(img):
    
    return hog(
        img,
        orientations=9,
        pixels_per_cell=(8,8),
        cells_per_block=(2,2),
        block_norm='L2-Hys',
        feature_vector=True
    )


In [ ]:
import cv2
import numpy as np

def extract_color_hist(img):
    
    img_resized = cv2.resize(img, TARGET_SIZE)
    hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0,1,2], None, (8,8,8),
                        [0,180, 0,256, 0,256])
    cv2.normalize(hist, hist)
    return hist.flatten()


#### Edge Histogram / Canny + Contour Features

In [ ]:
def extract_edge_ratio(img):
    
    img_resized = cv2.resize(img, TARGET_SIZE)
    edges = cv2.Canny(img_resized, 50,150)
    return np.array([edges.sum() / edges.size])


#### Combine

In [ ]:
import numpy as np
from skimage import io, color, transform

def extract_features(path):
    gray = io.imread(path, as_gray=True)
    gray = transform.resize(gray, TARGET_SIZE, anti_aliasing=True)
    
    color_img = cv2.imread(path, cv2.IMREAD_COLOR)
    
    f_hog  = extract_hog(gray)
    f_hist = extract_color_hist(color_img)
    f_edge = extract_edge_ratio((gray * 255).astype(np.uint8))
    
    return np.hstack([f_hog, f_hist, f_edge])


### Build Your Feature Matrix & Labels

In [6]:
from tqdm import tqdm

X = []
y = []
for _, row in df.iterrows():
    X.append(extract_features(row.filepath))
    y.append(row.label)

X = np.array(X)
y = np.array(y)


###  Train/Test Split & Scaling

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)


### Train a Classifier

#### SVM (not used)

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1,1,10],
    'gamma': ['scale','auto'],
    'kernel': ['rbf','linear']
}
svm = GridSearchCV(SVC(probability=True), param_grid, cv=5, verbose= 2, n_jobs=-1)
svm.fit(X_train, y_train)
print("Best SVM params:", svm.best_params_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


KeyboardInterrupt: 

#### XGboost (our model)

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dval   = xgb.DMatrix(X_test,  label=y_test)

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc'
}

bst = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    early_stopping_rounds=10,
    evals=[(dtrain, 'train'), (dval, 'val')],
    verbose_eval=True
)



[0]	train-auc:0.93373	val-auc:0.88746
[1]	train-auc:0.95876	val-auc:0.90692
[2]	train-auc:0.96888	val-auc:0.91895
[3]	train-auc:0.97518	val-auc:0.92541
[4]	train-auc:0.97906	val-auc:0.93093
[5]	train-auc:0.98257	val-auc:0.93808
[6]	train-auc:0.98549	val-auc:0.94310
[7]	train-auc:0.98854	val-auc:0.94616
[8]	train-auc:0.99098	val-auc:0.95051
[9]	train-auc:0.99309	val-auc:0.95320
[10]	train-auc:0.99452	val-auc:0.95536
[11]	train-auc:0.99591	val-auc:0.95724
[12]	train-auc:0.99684	val-auc:0.95852
[13]	train-auc:0.99767	val-auc:0.96025
[14]	train-auc:0.99846	val-auc:0.96199
[15]	train-auc:0.99881	val-auc:0.96278
[16]	train-auc:0.99910	val-auc:0.96347
[17]	train-auc:0.99936	val-auc:0.96451
[18]	train-auc:0.99950	val-auc:0.96553
[19]	train-auc:0.99967	val-auc:0.96583
[20]	train-auc:0.99977	val-auc:0.96724
[21]	train-auc:0.99985	val-auc:0.96776
[22]	train-auc:0.99987	val-auc:0.96861
[23]	train-auc:0.99994	val-auc:0.96908
[24]	train-auc:0.99997	val-auc:0.96954
[25]	train-auc:0.99997	val-auc:0.97

### Evaluate

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score


import xgboost as xgb
dtest      = xgb.DMatrix(X_test)
y_prob_xgb = bst.predict(dtest)
y_pred_xgb = (y_prob_xgb > 0.5).astype(int)

print("--- XGBoost (Booster) ---")
print(classification_report(y_test, y_pred_xgb))
print("ROC AUC:", roc_auc_score(y_test, y_prob_xgb))





--- XGBoost (Booster) ---
              precision    recall  f1-score   support

           0       0.95      0.92      0.93      1200
           1       0.92      0.95      0.93      1200

    accuracy                           0.93      2400
   macro avg       0.93      0.93      0.93      2400
weighted avg       0.93      0.93      0.93      2400

ROC AUC: 0.9827159722222222


In [2]:
bst = xgb.Booster()
bst.load_model('weapon_booster.model')

#### predict function (to run on some test cases)

In [ ]:
import cv2
import numpy as np
import xgboost as xgb  
from skimage.feature import hog
from skimage import io, transform

TARGET_SIZE = (128,128)

def extract_features(path):
    gray = io.imread(path, as_gray=True)
    gray = transform.resize(gray, TARGET_SIZE, anti_aliasing=True)
    
    hog_vec = hog(
        gray,
        orientations=9,
        pixels_per_cell=(8,8),
        cells_per_block=(2,2),
        block_norm='L2-Hys',
        feature_vector=True
    )
    
    color_img = cv2.imread(path, cv2.IMREAD_COLOR)
    color_resized = cv2.resize(color_img, TARGET_SIZE)
    hsv = cv2.cvtColor(color_resized, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0,1,2], None, (8,8,8),
                        [0,180,0,256,0,256]).flatten()
    cv2.normalize(hist, hist)
    
    edges = cv2.Canny((gray*255).astype(np.uint8), 50,150)
    edge_ratio = np.array([edges.sum()/edges.size])
    
    return np.hstack([hog_vec, hist, edge_ratio])


def predict_image(path, model, scaler=None, booster=bst, threshold=0.5):
    
    feats = extract_features(path).reshape(1, -1)
    
    if scaler is not None:
        feats = scaler.transform(feats)

    if booster is not None:
        
        dmat = xgb.DMatrix(feats)
        prob = booster.predict(dmat)[0]
        pred = int(prob > threshold)
    else:
        
        prob = model.predict_proba(feats)[0,1]
        pred = model.predict(feats)[0]

    label = "weapon" if pred==1 else "no_weapon"
    print(f"Prediction: {label} (p={prob:.3f})")
    return pred, prob


path = '/home/group02-f24/Documents/Khalil/CV/Traditional_CV/input_for_yolo_with_x/pexels-specna-arms-306304-886453.jpg'

pred, prob = predict_image(path, model=None, scaler=scaler, booster=bst)


Prediction: no_weapon (p=0.240)


#### cropping the gun photos

In [ ]:
import os
import json
from PIL import Image

images_dir = "/home/group02-f24/Documents/Khalil/CV/Traditional_CV/Guns/ds/img"
json_dir = "/home/group02-f24/Documents/Khalil/CV/Traditional_CV/Guns/ds/ann"
output_dir = "/home/group02-f24/Documents/Khalil/CV/Traditional_CV/Guns_cropped"

os.makedirs(output_dir, exist_ok=True)

def crop_image(image_path, json_path, output_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    img = Image.open(image_path)
    
    obj = data['objects'][0]
    exterior = obj['points']['exterior']
    
    left = int(exterior[0][0])
    upper = int(exterior[0][1])
    right = int(exterior[1][0])
    lower = int(exterior[1][1])
    
    cropped_img = img.crop((left, upper, right, lower))
    cropped_img.save(output_path)

for filename in os.listdir(images_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(images_dir, filename)
        
        json_filename = filename + '.json'
        json_path = os.path.join(json_dir, json_filename)
        
        if os.path.exists(json_path):
            output_path = os.path.join(output_dir, filename)
            crop_image(image_path, json_path, output_path)
            print(f"Cropped and saved {filename}")
        else:
            print(f"JSON file not found for {filename}")


Cropped and saved armas (554).jpg
Cropped and saved armas (385).jpg
Cropped and saved armas (2120).jpg
Cropped and saved armas (2490).jpg
Cropped and saved armas (2723).jpg
Cropped and saved armas (1380).jpg
Cropped and saved armas (665).jpg
Cropped and saved armas (2970).jpg
Cropped and saved armas (2522).jpg
Cropped and saved armas (2672).jpg
Cropped and saved armas (1404).jpg
Cropped and saved armas (634).jpg
Cropped and saved armas (2674).jpg
Cropped and saved armas (2400).jpg
Cropped and saved armas (463).jpg
Cropped and saved armas (2170).jpg
Cropped and saved armas (2991).jpg
Cropped and saved armas (2457).jpg
Cropped and saved armas (1962).jpg
Cropped and saved armas (1363).jpg
Cropped and saved armas (1637).jpg
Cropped and saved armas (2280).jpg
Cropped and saved armas (1155).jpg
Cropped and saved armas (719).jpg
Cropped and saved armas (1005).jpg
Cropped and saved armas (157).jpg
Cropped and saved armas (2488).jpg
Cropped and saved armas (987).jpg
Cropped and saved armas (164

#### preparing the data (2000 knife 2000 gun 2000 heavy weapon 6000 non)

In [ ]:
import os
import random
import shutil

def copy_random_images_append(src_folder, dst_folder, num_images=6000):
    os.makedirs(dst_folder, exist_ok=True)

    valid_exts = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
    all_images = [f for f in os.listdir(src_folder)
                  if os.path.splitext(f)[1].lower() in valid_exts]

    if len(all_images) < num_images:
        print(f"Warning: only found {len(all_images)} images, less than {num_images} requested.")
        num_images = len(all_images)

    selected_images = random.sample(all_images, num_images)

    for img_name in selected_images:
        src_path = os.path.join(src_folder, img_name)

        dst_path = os.path.join(dst_folder, img_name)

        base, ext = os.path.splitext(img_name)
        count = 1
        while os.path.exists(dst_path):
            dst_path = os.path.join(dst_folder, f"{base}_{count}{ext}")
            count += 1

        shutil.copy2(src_path, dst_path)

    print(f"Appended {num_images} images from {src_folder} to {dst_folder}.")

source_folder = "/home/group02-f24/Documents/Khalil/CV/negative_images/images"
destination_folder = "no_weapons"
copy_random_images_append(source_folder, destination_folder, 6000)


Appended 6000 images from /home/group02-f24/Documents/Khalil/CV/negative_images/images to no_weapons.


### Evaluate on YOLO validation set

In [11]:
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np


valid_images_dir = "/home/group02-f24/Documents/Khalil/CV/Traditional_CV/weapon-detection.v16-remapped-train-80-val-20.yolov11/valid/images"
valid_labels_dir = "/home/group02-f24/Documents/Khalil/CV/Traditional_CV/weapon-detection.v16-remapped-train-80-val-20.yolov11/valid/labels"

true_labels = []
pred_labels = []

image_files = [f for f in os.listdir(valid_images_dir) if f.lower().endswith(('.jpg','.png','.jpeg'))]

for img_file in image_files:
    img_path = os.path.join(valid_images_dir, img_file)
    label_file = os.path.splitext(img_file)[0] + ".txt"
    label_path = os.path.join(valid_labels_dir, label_file)

    gt = 0
    if os.path.exists(label_path):
        with open(label_path, 'r') as lf:
            lines = lf.readlines()
            for line in lines:
                if line.strip().startswith('0'):
                    gt = 1
                    break

    pred, prob = predict_image(img_path, model=None, scaler=scaler, booster=bst, threshold=0.5)

    true_labels.append(gt)
    pred_labels.append(pred)

acc = accuracy_score(true_labels, pred_labels)
prec = precision_score(true_labels, pred_labels)
rec = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)

print(f"Validation Results:\nAccuracy: {acc:.4f}\nPrecision: {prec:.4f}\nRecall: {rec:.4f}\nF1 Score: {f1:.4f}")


Prediction: weapon (p=0.986)
Prediction: weapon (p=0.994)
Prediction: no_weapon (p=0.162)
Prediction: weapon (p=0.623)
Prediction: no_weapon (p=0.000)
Prediction: weapon (p=1.000)
Prediction: weapon (p=0.846)
Prediction: weapon (p=1.000)
Prediction: weapon (p=1.000)
Prediction: weapon (p=0.988)
Prediction: no_weapon (p=0.020)
Prediction: no_weapon (p=0.002)
Prediction: no_weapon (p=0.000)
Prediction: no_weapon (p=0.093)
Prediction: weapon (p=1.000)
Prediction: no_weapon (p=0.006)
Prediction: weapon (p=0.996)
Prediction: weapon (p=1.000)
Prediction: no_weapon (p=0.447)
Prediction: no_weapon (p=0.000)
Prediction: weapon (p=0.999)
Prediction: weapon (p=1.000)
Prediction: weapon (p=0.738)
Prediction: no_weapon (p=0.002)
Prediction: no_weapon (p=0.000)
Prediction: no_weapon (p=0.001)
Prediction: weapon (p=0.997)
Prediction: weapon (p=1.000)
Prediction: no_weapon (p=0.003)
Prediction: no_weapon (p=0.027)
Prediction: weapon (p=1.000)
Prediction: weapon (p=0.644)
Prediction: weapon (p=0.986)
P

KeyboardInterrupt: 

### YOLO + Traditional CV (complete pipeline )

In [ ]:
import os
from pathlib import Path
from PIL import Image, ImageDraw
from ultralytics import YOLO
import cv2
import numpy as np
from skimage.feature import hog
from skimage import io, transform

# loud yolo

yolo = YOLO('/home/group02-f24/Documents/Khalil/CV/ultralytics/yolo11x.pt')

booster = bst
 
def process_folder(image_folder, output_folder, crop_folder):
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(crop_folder, exist_ok=True)

    for img_path in Path(image_folder).glob('*.*'):
        if img_path.suffix.lower() not in {'.jpg','.jpeg','.png','.bmp'}:
            continue

        pil_img = Image.open(img_path).convert('RGB')
        draw   = ImageDraw.Draw(pil_img)

        # detect objects
        results = yolo.predict(source=str(img_path), conf=0.3)

        for i, box in enumerate(results[0].boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            crop = pil_img.crop((x1, y1, x2, y2))

            # save crop permanently
            crop_name = f"{img_path.stem}_crop_{i}.jpg"
            crop_path = Path(crop_folder) / crop_name
            crop.save(crop_path)

            #  prediction
            pred, prob = predict_image(
                str(crop_path),
                model=None,
                scaler=scaler,
                booster=booster
            )

            # annotate original if weapon
            if pred == 1:
                draw.rectangle([x1, y1, x2, y2], outline='red', width=3)
                draw.text((x1, y1-10), f"{prob:.2f}", fill='red')

        # save annotated image
        out_path = Path(output_folder) / img_path.name
        pil_img.save(out_path)
        print(f"Processed {img_path.name}, crops saved to {crop_folder}")

if __name__ == '__main__':
    process_folder(
        image_folder  = '/home/group02-f24/Documents/Khalil/CV/Traditional_CV/input_for_yolo_with_x',
        output_folder = '/home/group02-f24/Documents/Khalil/CV/Traditional_CV/results',
        crop_folder   = '/home/group02-f24/Documents/Khalil/CV/Traditional_CV/crops'
    )



image 1/1 /home/group02-f24/Documents/Khalil/CV/Traditional_CV/input_for_yolo_with_x/pexels-specna-arms-306304-886453.jpg: 448x640 1 person, 1 motorcycle, 12.9ms
Speed: 1.7ms preprocess, 12.9ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)
Prediction: weapon (p=0.700)
Prediction: weapon (p=0.999)
Processed pexels-specna-arms-306304-886453.jpg, crops saved to /home/group02-f24/Documents/Khalil/CV/Traditional_CV/crops
